In [ ]:
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
from tensorflow.keras.utils import plot_model
import os
import wfdb
from tqdm import tqdm

ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.

ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.



Traceback (most recent call last):
  File "<frozen importlib._bootstrap_external>", line 87, in _path_is_mode_type
  File "<frozen importlib._bootstrap_external>", line 81, in _path_stat
FileNotFoundError: [WinError 2] The system cannot find the file specified: 'C:\\Users\\Jordan\\Anaconda3\\envs\\tf-NNN-build\\lib\\site-packages\\tensorflow\\python\\util\\__init__.cp37-win_amd64.pyd'

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "C:\Users\Jordan\Anaconda3\envs\tf-NNN-build\lib\site-packages\IPython\core\interactiveshell.py", line 3457, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "C:\Users\Jordan\AppData\Local\Temp/ipykernel_4760/1850367790.py", line 1, in <module>
    import tensorflow as tf
  File "C:\Users\Jordan\Anaconda3\envs\tf-NNN-build\lib\site-packages\tensorflow\__init__.py", line 28, in <module>
    from tensorflow.python import pywrap_tensorflow  # pylint: disable=unused-import
  

TypeError: object of type 'NoneType' has no len()

## Initialisation

In [ ]:
min_heartrate_expected = 30 # TODO potentially change this
sample_frequency = 500
min_RR_interval_length = 60/min_heartrate_expected
max_sample_length = sample_frequency * 5 * min_RR_interval_length 

max_sample_length = 18286

## Convolutional Block

In [ ]:
def convolutional_block(x, filter, rate, pool=False):
    
    # copy tensor to variable called x_skip
    x_skip = x
    if pool: x_skip = tf.keras.layers.MaxPool1D()(x_skip)
    
    x = tf.keras.layers.BatchNormalization()(x) #TODO check axis here
    x = tf.keras.layers.Activation('relu')(x)
    x = tf.keras.layers.Dropout(rate)(x)
    x = tf.keras.layers.Conv1D(filter, 15, padding='same', strides=(1))(x) # TODO change hyperparameters here
    if pool: x = tf.keras.layers.AveragePooling1D()(x)
    
    # add residue TODO potentially do this better with connections between multiple layers 
    x = tf.keras.layers.Add()([x, x_skip])     
    
    return x

## Total model build

In [ ]:
def NNN(shape = (max_sample_length, 1), classes = 3):
    
    # Setup Input Layer
    x_input = tf.keras.layers.Input(shape)
    # x = tf.keras.layers.ZeroPadding1D()(x_input) # TODO check if this padding is neccessary
    x = x_input
    
    # Define initial filter size
    filter_size = 15
    dropout_rate = 0.01
    
    # Build 16 convolutional blocks
    for i in range(16):
        print(i, x.shape)
        # The filter size will go on increasing by a factor of 2
        filter_size = 1
        x = convolutional_block(x, filter_size, dropout_rate, pool = (i%3 == 0)) # only pool one in every 3 layers
    
    # End Dense Network
    n_dense_parameters = 32
    x = tf.keras.layers.Flatten()(x) # TODO check if this is neccessary
    x = tf.keras.layers.Dense(n_dense_parameters, activation='relu')(x) # optimise over this n_dense_parameters variable
    x = tf.keras.layers.Dense(classes, activation='softmax')(x) 
    model = tf.keras.models.Model(inputs=x_input, outputs=x, name="NNNet")
    return model

In [ ]:
neural_network = NNN()

In [ ]:
# plot_model(neural_network, to_file='model_plot.png', show_shapes=True, show_layer_names=True)

### Compile the model

In [ ]:
neural_network.compile(optimizer='adam', loss='categorical_crossentropy') # TODO experiment with loss and optimizers

### Prepare the data for training

In [ ]:
from utils.metadata import find_maximum_time_length_of_sample_in_dataset
import pandas as pd

In [ ]:
dataset_path = "2_data\\physionet_datasets\\training2017"

file_list = [file_ for file_ in os.listdir(dataset_path) if file_.endswith('.mat')]
# batched_file_list = [file_list[x:x+500] for x in range(0, len(file_list), 500)] # batch to save memory

# frequence_of_samples = wfdb.rdrecord(os.path.join(dataset_path, file_list[0].split('.mat',2)[0]), channels=[0]).fs
# maximum_length_of_samples = int(find_maximum_time_length_of_sample_in_dataset(dataset_path) * frequence_of_samples)
# maximum_length_of_samples

In [ ]:
%store -r maximum_length_of_samples

In [ ]:
# data = np.zeros((len(file_list), int(maximum_length_of_samples)))
# for i, file in tqdm(enumerate(file_list)):
#     x = wfdb.rdrecord(os.path.join(dataset_path,file.split('.mat',2)[0]), channels=[0]).p_signal
#     data[i][:x.shape[0]] = np.squeeze(x) 
# np.save(f"{dataset_path}\\2017\\entire_dataset", np.array(data))

In [ ]:
ecg_data = np.load(f"{dataset_path}\\2017\\entire_dataset.npy", allow_pickle=True)
ecg_data.shape

In [ ]:
# import the labels
label_file = os.path.join(dataset_path, "REFERENCE.csv")
labels = pd.read_csv(label_file, header = None, names = ['name', 'label']).to_numpy()
labels.shape

### Remove noisy samples

In [ ]:
indices_no_noise = [i for i,label in enumerate(labels[:,1]) if label != '~']
ecg_data_no_noise = ecg_data[indices_no_noise]
labels_no_noise = labels[indices_no_noise]
len(indices_no_noise)

### Encode the labels

In [ ]:
data_arr_with_labels = np.zeros((len(indices_no_noise), 2), dtype=object)
print(data_arr_with_labels.shape, ecg_data.shape, np.expand_dims(labels[:,1],1).shape)

def generate_encoded_label(label):
    if 'N' in label: return [1,0,0]
    elif 'A' in label: return [0,1,0]
    elif 'O' in label: return [0,0,1]
    else: raise ValueError('Label given does not match any known label')

counter = 0
for i in tqdm(indices_no_noise):
    
    data_arr_with_labels[counter,0] = list(ecg_data[i])
    data_arr_with_labels[counter,1] = generate_encoded_label(labels[i,1])
    counter += 1
        
assert counter == len(indices_no_noise)

In [ ]:
column_names = ['ECG', 'Label']

dataset = pd.DataFrame(data = data_arr_with_labels, 
                  index = [label[0] for label in labels if label[1] != '~'], 
                  columns = column_names)

# dataset.to_csv(f"{dataset_path}\\2017\\entire_dataset.csv")


In [ ]:
dataset.head()

In [ ]:
ecg_data.shape

In [ ]:
import sys
import csv
maxInt = sys.maxsize

while True:
    # decrease the maxInt value by factor 10 
    # as long as the OverflowError occurs.

    try:
        csv.field_size_limit(maxInt)
        break
    except OverflowError:
        maxInt = int(maxInt/10)

In [ ]:
batch_size = 32

tf_dataset = tf.data.experimental.make_csv_dataset(f"{dataset_path}\\2017\\entire_dataset.csv", 
                                                batch_size = 1, 
                                                select_columns = ['ECG', 'Label'], 
                                                label_name = 'Label',
                                                num_epochs = 1)

In [ ]:
next(iter(tf_dataset))

### Train the model

In [ ]:
dataset.to_numpy().shape
dataset_arr = dataset.to_numpy()
dataset_arr[:,0].shape, dataset_arr[:,1].shape

In [ ]:
np.expand_dims(labels[:,1],1).shape, np.expand_dims(ecg_data,2).shape

In [ ]:
np.expand_dims(data_arr_with_labels[:,:-1],2).shape, data_arr_with_labels[:,-1].shape

In [ ]:
neural_network.fit(np.expand_dims(data_arr_with_labels[:,:-1],2), data_arr_with_labels[:,-1], epochs=10)

Is it better to build binary classification model between 'N' and 'AF or O' and then 'AF' or 'O'?